In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jan 18 15:28:30 2020

@author: Yuan
"""

import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk

# =============================================================================
# nltk.download('wordnet')
# =============================================================================
data = pd.read_csv('g.csv', error_bad_lines=False, encoding = "ISO-8859-1")
data_text = data[['title','simple description','description for Gccampus']]
documents = data_text

documents['title'] = documents['title'].fillna('999')
documents = documents.drop(documents[(documents.title=='999')].index.tolist())
documents = documents.drop(documents[(documents.title=='Cours de EFPC / CSPS courses')].index.tolist())
documents = documents.drop(documents[(documents.title=='Cours provenant d\'une autre ministÃ¨re / Other Government Department courses')].index.tolist())
documents = documents.drop(documents[(documents.title=='Externals Ressources Links')].index.tolist())
documents = documents.drop(documents[(documents.title=='Cours de Skillsoft courses')].index.tolist())
documents = documents.drop(documents[(documents.title=='Delegation of Authority - video')].index.tolist())
documents = documents.drop(documents[(documents.title=='Blogs')].index.tolist())
documents = documents.drop(documents[(documents.title=='Cases Studies')].index.tolist())
documents = documents.drop(documents[(documents.title=='hq')].index.tolist())
documents = documents.drop(documents[(documents.title=='Video List')].index.tolist())

for i in documents.index:
    if(' '.join(str(documents['simple description'][i]).split()[:10])==' '.join(str(documents['description for Gccampus'][i]).split()[:10])):
        documents['simple description'][i]= ""

documents.to_csv("clean.csv")

documents['combine'] = documents[['title','simple description','description for Gccampus']].apply(lambda x : '{} {} {}'.format(x[0],x[1],x[2]), axis=1)

# documents = documents['combine']

documents.to_csv("combine.csv")

# df.drop('name', axis=1)

stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


doc_sample = documents[documents.index == 4].values[0][0]
print('\noriginal document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\ntokenized and lemmatized document: ')
print(preprocess(doc_sample))
processed_docs = documents['combine'].map(preprocess)
print(documents['combine'].shape[0])
processed_docs = processed_docs[:documents['combine'].shape[0]]

# Bag of words on the dataset
dictionary = gensim.corpora.Dictionary(processed_docs)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

import pyLDAvis.gensim
import pyLDAvis
pyLDAvis.enable_notebook()

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=3)


from gensim.test.utils import datapath

lda_model.save("word2vec.model")

lda_model = gensim.models.ldamodel.LdaModel.load("word2vec.model")

pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

# for idx, topic in lda_model.print_topics(-1):
#     print('Topic: {} \nWords: {}'.format(idx, topic))


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



original document: 
['Being', 'Brief:', 'Making', 'a', 'Request', 'Efficiently', '(B196)']

tokenized and lemmatized document: 
['brief', 'make', 'request', 'effici']
1117
0 activ
1 appli
2 asset
3 awar
4 best
5 contribut
6 cours
7 department
8 direct
9 foundat
10 gain


//anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.019590  0.006518       1        1  16.951836
6      0.009133 -0.018753       2        1  13.644353
8     -0.039335 -0.000166       3        1  13.030265
0      0.009622 -0.069651       4        1  11.622942
4     -0.023584 -0.040661       5        1  11.583907
2     -0.025206 -0.053939       6        1   8.574123
3      0.124242  0.051552       7        1   8.216372
1      0.048080 -0.032055       8        1   6.635322
5      0.029008  0.080918       9        1   5.028734
9     -0.112371  0.076237      10        1   4.712147, topic_info=     Category        Freq         Term       Total  loglift  logprob
789   Default  164.000000        gcdoc  164.000000  30.0000  30.0000
787   Default   82.000000     document   82.000000  29.0000  29.0000
122   Default  168.000000     communic  168.000000  28.0000  28.0000
13    Default  204.000000       inform  204.000000  27.0000  27.0000
400   Default   66.000000       author   66.000000  26.0000  26.0000
29    Default  397.000000       servic  397.000000  25.0000  25.0000
150   Default  294.000000         work  294.000000  24.0000  24.0000
56    Default  294.000000       canada  294.000000  23.0000  23.0000
402   Default   61.000000        deleg   61.000000  22.0000  22.0000
380   Default   99.000000      financi   99.000000  21.0000  21.0000
156   Default   96.000000      languag   96.000000  20.0000  20.0000
121   Default  189.000000        chang  189.000000  19.0000  19.0000
645   Default  142.000000         team  142.000000  18.0000  18.0000
1418  Default   50.000000       custom   50.000000  17.0000  17.0000
25    Default  351.000000       public  351.000000  16.0000  16.0000
988   Default   33.000000       gender   33.000000  15.0000  15.0000
556   Default   49.000000       negoti   49.000000  14.0000  14.0000
11    Default  371.000000       govern  371.000000  13.0000  13.0000
41    Default  302.000000        learn  302.000000  12.0000  12.0000
593   Default   45.000000        board   45.000000  11.0000  11.0000
600   Default   43.000000     treasuri   43.000000  10.0000  10.0000
683   Default   84.000000       deputi   84.000000   9.0000   9.0000
1151  Default   86.000000       mental   86.000000   8.0000   8.0000
814   Default   32.000000     interfac   32.000000   7.0000   7.0000
811   Default   25.000000      version   25.000000   6.0000   6.0000
352   Default  173.000000     workplac  173.000000   5.0000   5.0000
815   Default   44.000000         view   44.000000   4.0000   4.0000
258   Default   57.000000       divers   57.000000   3.0000   3.0000
326   Default   42.000000        modul   42.000000   2.0000   2.0000
155   Default   37.000000     influenc   37.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
94    Topic10   10.462858         base   57.859512   1.3448  -5.0134
117   Topic10   10.340362         valu   57.573395   1.3380  -5.0252
25    Topic10   27.107737       public  351.894562   0.4915  -4.0614
697   Topic10    4.391523  partnership   13.086708   1.9631  -5.8816
409   Topic10   10.719867         open   73.858765   1.1250  -4.9892
505   Topic10    7.492299         idea   37.290321   1.4502  -5.3474
270   Topic10    7.334859     conflict   36.202934   1.4585  -5.3686
29    Topic10   24.436678       servic  397.925812   0.2648  -4.1652
41    Topic10   19.534149        learn  302.598816   0.3148  -4.3891
13    Topic10   16.109556       inform  204.455963   0.5141  -4.5818
378   Topic10    9.794010       cultur   73.974167   1.0331  -5.0795
352   Topic10   13.946072     workplac  173.788193   0.5324  -4.7261
144   Topic10    8.584795        share   60.216255   1.1071  -5.2113
744   Topic10    8.986827        innov   69.776306   1.0055  -5.1655
310   Topic10    6.517633     guidelin   35.081650   1.3719  -5.4867
1151  Topic10    8.647596       mental   86.473778   0.7525  -5.20